In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import env
from sqlalchemy import text, create_engine

In [2]:
logs_query = """select * 
                from api_access"""

url = env.get_db_url('logs')
    
df = pd.read_sql(logs_query, url)

In [3]:
df.head()

,entry
0,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
1,97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] ...
2,97.105.19.58 - - [16/Apr/2019:19:34:44 +0000] ...
3,97.105.19.58 - - [16/Apr/2019:19:34:46 +0000] ...
4,97.105.19.58 - - [16/Apr/2019:19:34:48 +0000] ...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13974 entries, 0 to 13973
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   entry   13974 non-null  object
dtypes: object(1)
memory usage: 109.3+ KB


### Prepare
* the data needs to be parsed out so we can see each piece of the access log

In [5]:
df.entry[0]

'97.105.19.58 - - [16/Apr/2019:19:34:42 +0000] "GET /api/v1/sales?page=81 HTTP/1.1" 200 512495 "-" "python-requests/2.21.0"'

In [6]:
#separating the elements in one row
df.entry[0].split()

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [7]:
#save it
parts = df.entry[0].split()
parts

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [8]:
df.entry.str.split(expand=True).head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,97.105.19.58,-,-,[16/Apr/2019:19:34:42,+0000],"""GET",/api/v1/sales?page=81,"HTTP/1.1""",200,512495,...,None,None,None,None,None,None,None,None,None,None
1,97.105.19.58,-,-,[16/Apr/2019:19:34:42,+0000],"""GET",/api/v1/items,"HTTP/1.1""",200,3561,...,None,None,None,None,None,None,None,None,None,None
2,97.105.19.58,-,-,[16/Apr/2019:19:34:44,+0000],"""GET",/api/v1/sales?page=82,"HTTP/1.1""",200,510103,...,None,None,None,None,None,None,None,None,None,None
3,97.105.19.58,-,-,[16/Apr/2019:19:34:46,+0000],"""GET",/api/v1/sales?page=83,"HTTP/1.1""",200,510003,...,None,None,None,None,None,None,None,None,None,None
4,97.105.19.58,-,-,[16/Apr/2019:19:34:48,+0000],"""GET",/api/v1/sales?page=84,"HTTP/1.1""",200,511963,...,None,None,None,None,None,None,None,None,None,None


In [9]:
#create dictionary to hold results
output= {}

In [10]:
#pull out ip address
output['ip'] = parts[0]

In [11]:
output

{'ip': '97.105.19.58'}

In [12]:
parts

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [13]:
df.entry.str.split(expand=True)[1].unique()

array(['-'], dtype=object)

In [14]:
df.entry.str.split(expand=True)[2].unique()

array(['-'], dtype=object)

In [15]:
#pull out time
output['time'] = parts[3][1:].replace(":",' ', 1)

In [16]:
output

{'ip': '97.105.19.58', 'time': '16/Apr/2019 19:34:42'}

In [17]:
parts

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [18]:
df.entry.str.split(expand=True)[4].unique()

array(['+0000]'], dtype=object)

In [19]:
#pull out request method
df.entry.str.split(expand=True)[5].unique()

array(['"GET'], dtype=object)

In [20]:
output['request_method'] = parts[5].strip('"')

In [21]:
output

{'ip': '97.105.19.58', 'time': '16/Apr/2019 19:34:42', 'request_method': 'GET'}

In [22]:
parts

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [23]:
#pull out request path
output['request_path'] = parts[6]

In [24]:
output

{'ip': '97.105.19.58',
 'time': '16/Apr/2019 19:34:42',
 'request_method': 'GET',
 'request_path': '/api/v1/sales?page=81'}

In [25]:
#pull our http version
output['http_version'] = parts[7][:-1]

In [26]:
output

{'ip': '97.105.19.58',
 'time': '16/Apr/2019 19:34:42',
 'request_method': 'GET',
 'request_path': '/api/v1/sales?page=81',
 'http_version': 'HTTP/1.1'}

In [27]:
parts

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [28]:
#pull out status code
output['status_code'] = parts[8]

In [29]:
output

{'ip': '97.105.19.58',
 'time': '16/Apr/2019 19:34:42',
 'request_method': 'GET',
 'request_path': '/api/v1/sales?page=81',
 'http_version': 'HTTP/1.1',
 'status_code': '200'}

In [30]:
# pull out size
output['size'] = parts[9]

In [31]:
output

{'ip': '97.105.19.58',
 'time': '16/Apr/2019 19:34:42',
 'request_method': 'GET',
 'request_path': '/api/v1/sales?page=81',
 'http_version': 'HTTP/1.1',
 'status_code': '200',
 'size': '512495'}

In [32]:
for col in df.entry.str.split(expand=True).iloc[:,11:].columns:
    print(df.entry.str.split(expand=True)[col].unique())

['"python-requests/2.21.0"' '"Mozilla/5.0' '"Slackbot-LinkExpanding'
 '"Slackbot' '"python-requests/2.20.1"' '"Python-urllib/3.7"']
[None '(Macintosh;' '1.0']
[None 'Intel' '(+https://api.slack.com/robots)"']
[None 'Mac']
[None 'OS']
[None 'X']
[None '10_14_4)' '10_14_3)' '10.14;']
[None 'AppleWebKit/537.36' 'rv:66.0)']
[None '(KHTML,' 'Gecko/20100101']
[None 'like' 'Firefox/66.0"']
[None 'Gecko)']
[None 'Chrome/73.0.3683.103' 'Chrome/72.0.3626.121']
[None 'Safari/537.36"']


In [33]:
#combine request agents together
' '.join(parts[11:]).strip('"')

'python-requests/2.21.0'

In [34]:
parts

['97.105.19.58',
 '-',
 '-',
 '[16/Apr/2019:19:34:42',
 '+0000]',
 '"GET',
 '/api/v1/sales?page=81',
 'HTTP/1.1"',
 '200',
 '512495',
 '"-"',
 '"python-requests/2.21.0"']

In [35]:
# # function to deal with parsing one entry in our log data
#this function works on one string
def parse_log_entry(entry):
    parts = entry.split()
    output = {}
    output['ip'] = parts[0]
    output['timestamp'] = parts[3][1:].replace(':', ' ', 1)
    output['request_method'] = parts[5][1:]
    output['request_path'] = parts[6]
    output['http_version'] = parts[7][:-1]
    output['status_code'] = parts[8]
    output['size'] = int(parts[9])
    output['user_agent'] = ' '.join(parts[11:]).strip('"')
    return pd.Series(output)

In [36]:
#apply string function to every element in a series
df = df.entry.apply(parse_log_entry)

In [37]:
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [38]:
# adding some anomalies
new = pd.DataFrame([
    ["95.31.18.119", "21/Apr/2019 10:02:41", "GET", "/api/v1/items/", "HTTP/1.1", 
     '200', 1153005, "python-requests/2.21.0"],
    ["95.31.16.121", "17/Apr/2019 19:36:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", 
     '301', 1005, "python-requests/2.21.0"],
    ["97.105.15.120", "18/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", 
     '301', 2560, "python-requests/2.21.0"],
    ["97.105.19.58", "19/Apr/2019 19:42:41", "GET", "/api/v1/sales?page=79/", "HTTP/1.1", 
     '200', 2056327, "python-requests/2.21.0"],
], columns=df.columns)

In [39]:
df = pd.concat([df, new])

In [40]:
df

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0
...,...,...,...,...,...,...,...,...
13973,97.105.19.58,17/Apr/2019 12:55:14,GET,/api/v1/sales?page=71,HTTP/1.1,200,510166,python-requests/2.21.0
0,95.31.18.119,21/Apr/2019 10:02:41,GET,/api/v1/items/,HTTP/1.1,200,1153005,python-requests/2.21.0
1,95.31.16.121,17/Apr/2019 19:36:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,1005,python-requests/2.21.0
2,97.105.15.120,18/Apr/2019 19:42:41,GET,/api/v1/sales?page=79/,HTTP/1.1,301,2560,python-requests/2.21.0


In [41]:
df = df.reset_index(drop=True)

In [42]:
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0


In [43]:
#divide by 1024 twice
df['size_mb'] = df['size'] / 1024 / 1024

In [44]:
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent,size_mb
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246


### Detecting Anomalies in Discrete Variables

* count
* frequency
* visualize
* conditional probability

### Count
* let's look at user_agent

# use replace() to rename the rows containing the string
df['user'] = df['user_agent'].replace('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36', 'Mozilla 1')
df['user'] = df['user_agent'].replace('Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0', 'Mozilla 2')
df['user'] = df['user_agent'].replace('Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)', 'Slackbot 1')
df['user'] = df['user_agent'].replace('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36', 'Mozilla 3')
df['user'] = df['user_agent'].replace('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36', 'Mozilla 4')
df['user'] = df['user_agent'].replace('Slackbot 1.0 (+https://api.slack.com/robots)', 'Slackbot 2')

In [47]:
# use replace() to rename the rows containing the strings
df['user'] = df['user_agent'].replace('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36', 'Mozilla 1') \
    .replace('Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0', 'Mozilla 2') \
    .replace('Slackbot-LinkExpanding 1.0 (+https://api.slack.com/robots)', 'Slackbot 1') \
    .replace('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36', 'Mozilla 3') \
    .replace('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36', 'Mozilla 4') \
    .replace('Slackbot 1.0 (+https://api.slack.com/robots)', 'Slackbot 2')

In [48]:
df.head()

,ip,timestamp,request_method,request_path,http_version,status_code,size,user_agent,size_mb,user
0,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/sales?page=81,HTTP/1.1,200,512495,python-requests/2.21.0,0.488753,python-requests/2.21.0
1,97.105.19.58,16/Apr/2019 19:34:42,GET,/api/v1/items,HTTP/1.1,200,3561,python-requests/2.21.0,0.003396,python-requests/2.21.0
2,97.105.19.58,16/Apr/2019 19:34:44,GET,/api/v1/sales?page=82,HTTP/1.1,200,510103,python-requests/2.21.0,0.486472,python-requests/2.21.0
3,97.105.19.58,16/Apr/2019 19:34:46,GET,/api/v1/sales?page=83,HTTP/1.1,200,510003,python-requests/2.21.0,0.486377,python-requests/2.21.0
4,97.105.19.58,16/Apr/2019 19:34:48,GET,/api/v1/sales?page=84,HTTP/1.1,200,511963,python-requests/2.21.0,0.488246,python-requests/2.21.0


In [49]:
df.groupby(['ip', 'user']).size()

ip              user                  
173.173.113.51  python-requests/2.21.0     1059
24.26.242.9     python-requests/2.21.0       21
3.88.129.158    Slackbot 1                    1
3.92.201.136    Slackbot 1                    1
34.207.64.242   Slackbot 2                    1
34.229.70.250   Slackbot 2                    1
35.174.209.2    Slackbot 2                    1
35.175.171.137  Slackbot 1                    2
45.23.250.16    python-requests/2.21.0        1
52.87.230.102   Slackbot 1                    1
                Slackbot 2                    1
52.90.165.200   Slackbot 2                    1
52.91.30.150    Slackbot 1                    1
54.145.52.184   Slackbot 2                    1
54.172.14.223   Slackbot 1                    1
68.201.219.223  python-requests/2.21.0       21
70.121.214.34   Mozilla 4                     2
72.181.105.81   Mozilla 1                     8
                Mozilla 2                     8
                python-requests/2.21.0      230
7

In [50]:
#probability of ip and status code
p_a_and_b = df.groupby(['ip','user']).size() / len(df)
p_a_and_b

ip              user                  
173.173.113.51  python-requests/2.21.0    0.075762
24.26.242.9     python-requests/2.21.0    0.001502
3.88.129.158    Slackbot 1                0.000072
3.92.201.136    Slackbot 1                0.000072
34.207.64.242   Slackbot 2                0.000072
34.229.70.250   Slackbot 2                0.000072
35.174.209.2    Slackbot 2                0.000072
35.175.171.137  Slackbot 1                0.000143
45.23.250.16    python-requests/2.21.0    0.000072
52.87.230.102   Slackbot 1                0.000072
                Slackbot 2                0.000072
52.90.165.200   Slackbot 2                0.000072
52.91.30.150    Slackbot 1                0.000072
54.145.52.184   Slackbot 2                0.000072
54.172.14.223   Slackbot 1                0.000072
68.201.219.223  python-requests/2.21.0    0.001502
70.121.214.34   Mozilla 4                 0.000143
72.181.105.81   Mozilla 1                 0.000572
                Mozilla 2                 0

In [53]:
#probability of ip
p_b = df.groupby('ip').size() / len(df)
p_b

ip
173.173.113.51    0.075762
24.26.242.9       0.001502
3.88.129.158      0.000072
3.92.201.136      0.000072
34.207.64.242     0.000072
34.229.70.250     0.000072
35.174.209.2      0.000072
35.175.171.137    0.000143
45.23.250.16      0.000072
52.87.230.102     0.000143
52.90.165.200     0.000072
52.91.30.150      0.000072
54.145.52.184     0.000072
54.172.14.223     0.000072
68.201.219.223    0.001502
70.121.214.34     0.000143
72.181.105.81     0.017599
72.181.113.170    0.043855
95.31.16.121      0.000072
95.31.18.119      0.000072
97.105.15.120     0.000072
97.105.19.58      0.858420
dtype: float64

In [54]:
p_a_and_b / p_b

ip              user                  
173.173.113.51  python-requests/2.21.0    1.000000
24.26.242.9     python-requests/2.21.0    1.000000
3.88.129.158    Slackbot 1                1.000000
3.92.201.136    Slackbot 1                1.000000
34.207.64.242   Slackbot 2                1.000000
34.229.70.250   Slackbot 2                1.000000
35.174.209.2    Slackbot 2                1.000000
35.175.171.137  Slackbot 1                1.000000
45.23.250.16    python-requests/2.21.0    1.000000
52.87.230.102   Slackbot 1                0.500000
                Slackbot 2                0.500000
52.90.165.200   Slackbot 2                1.000000
52.91.30.150    Slackbot 1                1.000000
54.145.52.184   Slackbot 2                1.000000
54.172.14.223   Slackbot 1                1.000000
68.201.219.223  python-requests/2.21.0    1.000000
70.121.214.34   Mozilla 4                 1.000000
72.181.105.81   Mozilla 1                 0.032520
                Mozilla 2                 0

In [55]:
# Given a particular ip, what is probability of a certain status code
user_given_ip = df.groupby('ip').user.value_counts(normalize=True).rename('prob').reset_index()
user_given_ip

,ip,user,prob
0,173.173.113.51,python-requests/2.21.0,1.000000
1,24.26.242.9,python-requests/2.21.0,1.000000
2,3.88.129.158,Slackbot 1,1.000000
3,3.92.201.136,Slackbot 1,1.000000
4,34.207.64.242,Slackbot 2,1.000000
5,34.229.70.250,Slackbot 2,1.000000
6,35.174.209.2,Slackbot 2,1.000000
7,35.175.171.137,Slackbot 1,1.000000
8,45.23.250.16,python-requests/2.21.0,1.000000
9,52.87.230.102,Slackbot 1,0.500000


In [56]:
user_given_ip [user_given_ip.prob < 1]

,ip,user,prob
9,52.87.230.102,Slackbot 1,0.500000
10,52.87.230.102,Slackbot 2,0.500000
17,72.181.105.81,python-requests/2.21.0,0.934959
18,72.181.105.81,Mozilla 1,0.032520
19,72.181.105.81,Mozilla 2,0.032520
20,72.181.113.170,python-requests/2.21.0,0.996737
21,72.181.113.170,Mozilla 3,0.003263
25,97.105.19.58,python-requests/2.21.0,0.838320
26,97.105.19.58,python-requests/2.20.1,0.159263
27,97.105.19.58,Mozilla 1,0.002167
